<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
</font>
</h1>
<h1 align=center>
 Texture Synthesis
</h2>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
توجه 1 : در این نوت بوک از فونت vazir برای بیان توضیحات استفاده شده است ، که لینک نصب این فونت در ادامه آمده است.

[vazir font download link](https://quera.ir/qbox/view/jLCUyovUmo/vazir-font-v30.1.0.zip)
</font>
</p>

<hr>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در ابتدا کتابخانه های موردنیاز را لود می کنیم 
</font>
</p>

In [ ]:
# import libraries:
import numpy as np
import matplotlib.pyplot as plt
import cv2

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
تابع <code>find_min_cut</code>:
<br>
در این تابع با استفاده از شیوه ی برنامه نویسی پویا (DP) مسیری که در روی آن مسیر کمترین اختلاف بین شدت روشنایی پیکسل های دو نوار را دارد ، به دست می آوریم.
<br>
این تابع ابتدا ماتریس <code>ssd</code> دو نواری که با هم تداخل دارند را ورودی می گیرد ، سپس با استفاده از ابعاد این ورودی عمودی یا افقی بودن آن را تشخیص می دهد و سپس بهینه ترین مسیر را انتخاب می کند.
<br>
الگوریتم DP ای که در این تابع به کار رفته است کاملا بر اساس گفته های کلاس و اسلاید 39 جلسه ی 12 می باشد که در ادامه شیوه ی یافتن مسیر بهینه در ورودی عمودی را شرح می دهیم ، هم چنین شیوه ی یافتن مسیر بهینه در ورودی افقی کاملا مشابه با عمودی می باشد که تنها در جزییاتی متفاوت است.
<br>
شیوه ی یافتن مسیر بهینه در حالت ورودی عمودی :
<br>
در ابتدا دو ماتریس با مقادیر تمام صفر با نام های <code>dp_cost</code> و <code>dp_next_col</code> را با ابعادی برابر با ابعاد ورودی (که در حالت پیش فرض 30*120) می باشد، مقدار دهی اولیه می کنیم.
<br>
با توجه به اینکه ورودی ما عمودی می باشد ، لذا درایه ی i ام ماتریس <code>dp_next_col</code> بیانگر آن است که اگر در سطر i ام نوار ورودی بودیم در سطر بعدی کدام ستون در مسیر بهینه ی ما قرار دارد و هم چنین ماتریس <dp_next_col> بیانگر آن است که اگر در درایه ی (i, j)بودیم ، هزینه ی بهترین مسیر ممکن با شروع از پایین ترین سطر ورودی و رسیدن به درایه ی (i, j) چقدر می باشد.
<br>
در نتیجه برای کامل کردن ماتریس های مذکور ابتدا آخرین سطر ماتریس <code>dp_cost</code> را عینا برابر با آخرین سطر ورودی (که خود ssd می باشد) قرار می دهیم و هم چنین آخرین سطر ماتریس <dp_next_col> را برابر با صفر می گذاریم.
<br>
سپس با استفاده از یک حلقه از سطر یکی مونده به پایان شروع می کنیم و درایه به درایه ماتریس های مذکور را کامل می کنیم.
<br>
حال برای به دست آوردن مسیر بهینه با کمترین هزینه ای که در سطر اول ماتریس <dp_cost> قرار دارد را پیدا می کنیم و در واقع ایندکس ستونی که کمترین هزینه را دارد، نشاندهنده ی شروع مسیر می باشد ، حال با به دست آوردن شروع مسیر و استفاده از ماتریس <code>dp_next_col</code> سایر ایندکس ستون ها به ازای هر ردیف بعدی را به دست آورده و این انیدکس ها را در ماتریس <code>min_cut_ind</code> ذخیره کرده و خروجی می دهیم.
<br>
نحوه ی پیاده سازی برای ورودی افقی نیز کاملا مشابه است ، با تفاوت های کوچک برای مثال،  برای حالت افقی به جای ماتریس <code>dp_nex_col</code> ماتریسی با نام <code>dp_next_row</code> خواهیم داشت که در آن انیدکس سطر بهینه به جای ستون بهینه نوشته شده است .
</font>
</p>

In [ ]:
def find_min_cut(input_ssd):
    ssd_h, ssd_w = input_ssd.shape
    direction = 'vertical'
    if ssd_h < ssd_w:
        direction = 'horizontal'

    if direction == 'vertical':
        dp_cost = np.zeros((ssd_h, ssd_w))
        dp_next_col = np.zeros((ssd_h, ssd_w))

        # initial last row of dp_cost = cost with dp_next_col = 0
        for i in range(ssd_w):
            dp_cost[ssd_h-1, i] = input_ssd[ssd_h-1, i]
            dp_next_col[ssd_h-1, i] = 0

        # fill other rows of dp_cost and dp_next_col:
        for i in np.arange(ssd_h-2, -1, -1):
            for j in range(ssd_w):
                min_cost = np.inf
                min_cost_ind = 0
                for k in range(max(0,j-1), min(j+2,ssd_w)):
                    if dp_cost[i+1, k] < min_cost:
                        min_cost = dp_cost[i+1, k]
                        min_cost_ind = k

                dp_cost[i, j] = input_ssd[i, j] + min_cost
                dp_next_col[i, j] = min_cost_ind

        # find minimum cut(index of each row that makes minimum cost):
        # produce min_cut_ind:
        min_cut_ind = []

        # fill min_cut_ind with find first columns:
        minimum_cost = np.amin(dp_cost[0, :]) 
        min_cut_ind.append(np.where(dp_cost[0, :] == minimum_cost)[0][0])

        for i in range(0, ssd_h - 1):
            min_cut_ind.append(int(dp_next_col[i, min_cut_ind[i]]))

    if direction == 'horizontal':
        dp_cost = np.zeros((ssd_h, ssd_w))
        dp_next_row = np.zeros((ssd_h, ssd_w))

        # initial last column of dp_cost = cost with dp_next_row = 0
        for i in range(ssd_h):
            dp_cost[i, ssd_w-1] = input_ssd[i, ssd_w-1]
            dp_next_row[i, ssd_w-1] = 0

        # fill other columns of dp_cost and dp_next_row:
        for j in np.arange(ssd_w-2, -1, -1): 
            for i in range(ssd_h):   
                min_cost = np.inf
                min_cost_ind = 0
                for k in range(max(0,i-1), min(i+2,ssd_h)):
                    if dp_cost[k, j+1] < min_cost:
                        min_cost = dp_cost[k, j+1]
                        min_cost_ind = k

                dp_cost[i, j] = input_ssd[i, j] + min_cost
                dp_next_row[i, j] = min_cost_ind

        # find minimum cut(index of each column that makes minimum cost):
        # produce min_cut_ind:
        min_cut_ind = []

        # fill min_cut_ind with find first row:
        minimum_cost = np.amin(dp_cost[:, 0]) 
        min_cut_ind.append(np.where(dp_cost[:, 0] == minimum_cost)[0][0])

        for j in range(0, ssd_w - 1):
            min_cut_ind.append(int(dp_next_row[min_cut_ind[j], j]))



    return min_cut_ind

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در تابع <code>find_joint_strip</code> ابتدا یک نوار از بافت سنتز شده و یک نواری که با استفاده از <code>template matching</code> یکی از شبیه ترین ها با به نوار سنتزشده می باشد را ورودی گرفته و هم چنین با ورودی گرفتن <code> min_cut_ind </code> ترکیبی از دو لبه را به صورتی که این دو لبه در مسیر بهینه ی min_cut به هم پیوند خورده اند را به دست می آورد و خروجی می دهد.
</font>
</p>

In [ ]:
def find_joint_strip(synthesized_strip, org_strip, min_cut_ind):
    strip_h, strip_w, _ = synthesized_strip.shape
    direction = 'vertical'
    if strip_h < strip_w:
        direction = 'horizontal'
    
    if direction == 'vertical':
        joint_strip = np.zeros((strip_h, strip_w, 3))
        for i in range(strip_h):
            for j in range(strip_w):
                if(j <= min_cut_ind[i]):
                    joint_strip[i, j, :] = synthesized_strip[i, j, :]
                else:
                    joint_strip[i, j, :] = org_strip[i, j, :]

    if direction == 'horizontal':
        joint_strip = np.zeros((strip_h, strip_w, 3))
        for j in range(strip_w):
            for i in range(strip_h):
                if(i <= min_cut_ind[j]):
                    joint_strip[i, j, :] = synthesized_strip[i, j, :]
                else:
                    joint_strip[i, j, :] = org_strip[i, j, :]


    return joint_strip
        

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
تابع <code>texture_synthesize</code>،
این تابع ورودی های زیر را می گیرد:
<br>
1. بافت اصلی (که اندازه ی کوچک تری نسبت به بافت سنتر شده دارد)
<br>
2. اندازه ی بعد بافت سنتز شده که به صورت مربعی می باشد(به صورت پیش فرض 2500 در نظر گرفته می شود)
<br>
3. اندازه ی patch ای که در شروع ساخت بافت سنتز شده به صورت کامل و در مراحل بعدی به صورتی که به همراه آن لبه باشد ، از بافت اصلی برمی داریم.(به طور پیش فرض برابر با 120 درنظر گرفته شده است)
<br>
عرض لبه ی ای که از patch ها بر می داریم. (قاعدتا طول لبه برابر با سایز  patch می باشد) (به صورت پیش فرض برابر با 30 در نظر گرفته می شود)
<br>
مراحل پیاده سازی این تابع به صورت زیر می باشد:
<br>
1. در ابتدا با استفاده از سایز های مختلف ورودی سایز تصویر سنتز شده ی نهایی مربعی را به دست می آوریم (این سایز ممکن است که با سایز خروجی ورودی بیشتر باشد ، مشکلی نیست در پایان کار با کراپ کردن بافت سنتز شده ی نهایی سایز آن را مطابق سایز در خواست شده می کنیم.)
<br>
2.در اولین مرتبه یک patch به صورت کاملا رندم از بافت اصلی برمی داریم و به عنوان اولین patch در بافت سنتز شده قرار می دهیم.
<br>
3.حال با استفاده از یک حلقه تمام بافت سنتز شده را تشکیل میدهیم ، نحوه ی این کار مطابق با روش گفته شده در کلاس و مطابق با اسلایدهای 23 تا 33 انجام می شود.
<br>
ابتدا ردیف اول را می سازیم (در حلقه هایی که مقدار i آن بین 1 تا patch_count می باشد ) به این صورت که سمت راست ترین نواری که تا به حال سنتز شده به دست می آوریم ، سپس با استفاده از تابع <code>cv2.matchTemplate</code> و هم چنین با روش ssd به صورت رندم یکی از 10 نواری که در بافت  اصلی به آن نوار نزدیک تر است ، پیدا می کنیم(محدوده ی سرچ را به گونه ای در نظر می گیریم که سمت راست آن نوار منتخب در بافت اصلی وجود داشته باشد )
؛ سپس اختلاف به صورت ssd آن دونوار را به دست آورده و آن را به تابع <code>find_min_cut</code> می دهیم تا مسیر بهینه ی بین آن دو نوار را به دست آوریم ، و در نهایت با به دست آمدن آن مسیر بهینه و استفاده از تابع <code>find_joint_strip</code> ، به هم پیوسته شده ی آن نوار به دست آورده و لذا از این به بعد یک نوار ترکیب شده خواهیم داشت.
<br>
حال این نوار ترکیب شده را جایگزین نوار برداشته شده از بافت سنتز شده کرده و سمت چپ نوار برداشته شده از بافت اصلی را نیز در ادامه ی بافت سنتز شده قرار می دهیم تا این patch افقی را به صورت کامل بسازیم.
<br>
عملیات گفته شده در بالا را آن قدر ادامه می دهیم تا ردیف اول به صورت کامل در بافت سنتزشده ساخته شود.
<br>
4.سپس برای ساخت patch های ستون اول بافت سنتز شده مشابه با ساخت patch های ردیف اول عمل می کنیم با این تفاوت که لبه ها در این حالت افقی هستند و به جای انتخاب راست ترین نوار از بافت ساخته شده تا به حال ، پایین ترین نوار ساخته شده تا به حال را انتخاب می کنیم و ...
<br>
5. و برای سنتز سایر patch ها (که نه در ردیف اول و نه در ستون اول هستند)هر دو عملیات مراحل 3 و 4 را انجام می دهیم چرا که لبه ها به صورت L شکل می شوند و لذا هم لبه ی به هم پیوسته ی افقی و هم لبه ی به هم پیوسته ی عمودی را به دست می آوریم و جایگزین می کنیم.
</font>
</p>

In [ ]:
def texture_synthesize(org_texture, output_size = 2500, patch_size = 120, strip_width = 30):
    # determine length and width of patch and strip according to inputs:
    patch_h, patch_w = patch_size, patch_size
    strip_h , strip_w = patch_size, strip_width

    # find patch_count in each row of synthesized_texture according to output_size:
    patch_count = int(np.ceil((output_size - strip_w) / (patch_h - strip_w)))
    synthesized_size = (patch_size - strip_width)*patch_count + strip_w
    synthesized_texture = np.zeros((synthesized_size, synthesized_size,3))

    # find first patch of synthesized_texture randomly:
    org_texture_h, org_texture_w, _ = org_texture.shape
    start_h = np.random.randint(0, org_texture_h-patch_h)
    start_w = np.random.randint(0, org_texture_w-patch_w)
    synthesized_texture[0:patch_h, 0:patch_w] = org_texture[start_h:start_h+patch_h, start_w:start_w+patch_w]

    # Synthesis other patchs of synthesized_texture:
    for i in range(1, int(patch_count**2)):
        # Synthesis first row of synthesized_texture:
        if(i // patch_count == 0):
            synthesized_strip_start_h = 0
            synthesized_strip_start_w = (patch_w - strip_w) * (i % int(patch_count))
            synthesized_strip_right = synthesized_texture[0:strip_h, synthesized_strip_start_w:synthesized_strip_start_w+strip_w]

            # use template matching with SSD method to find 10 most similar strips to synthesized_strip_right
            ssd_results = cv2.matchTemplate(np.uint8(org_texture[:,0:org_texture_w-(strip_h-strip_w)]), np.uint8(synthesized_strip_right), cv2.TM_SQDIFF)
            rand_ind = np.random.randint(1, 10)
            ssd_select = np.sort(ssd_results.flatten())[rand_ind]
            ssd_ind = (np.where(ssd_results == ssd_select)[0][0], np.where(ssd_results == ssd_select)[1][0])
            org_strip_left = org_texture[ssd_ind[0]:ssd_ind[0]+strip_h, ssd_ind[1]:ssd_ind[1]+strip_w]

            # find ssd matrix between synthesized_strip_right and org_strip_left
            org_strip_left_gray = cv2.cvtColor(org_strip_left, cv2.COLOR_BGR2GRAY)
            synthesized_strip_right_gray = cv2.cvtColor(np.uint8(synthesized_strip_right), cv2.COLOR_BGR2GRAY)
            strip_ssd = (org_strip_left_gray - synthesized_strip_right_gray) ** 2
                            

            # find vertically minimum cut for strip_ssd:
            min_cut_ind = find_min_cut(strip_ssd)
            joint_strip = find_joint_strip(synthesized_strip_right, org_strip_left, min_cut_ind)

            # replace joint_strip with synthesized_strip_right:
            synthesized_texture[0:strip_h, synthesized_strip_start_w:synthesized_strip_start_w+strip_w] = joint_strip
            synthesized_texture[0:patch_h, synthesized_strip_start_w+strip_w:synthesized_strip_start_w+patch_w] = (
            org_texture[ssd_ind[0]:ssd_ind[0]+patch_h, ssd_ind[1]+strip_w:ssd_ind[1]+patch_w]
            )

        # Synthesis first column of synthesized_texture:
        elif(i % patch_count == 0):
            synthesized_strip_start_h = int((patch_h - strip_w) * (i // patch_count))
            synthesized_strip_start_w = 0
            synthesized_strip_down = synthesized_texture[synthesized_strip_start_h:synthesized_strip_start_h+strip_w, 0:synthesized_strip_start_w+strip_h]

            # use template matching with SSD method to find 10 most similar strips to synthesized_strip_down
            ssd_results = cv2.matchTemplate(np.uint8(org_texture[0:org_texture_h-(strip_h-strip_w),:]), np.uint8(synthesized_strip_down), cv2.TM_SQDIFF)
            rand_ind = np.random.randint(1, 10)
            ssd_select = np.sort(ssd_results.flatten())[rand_ind]
            ssd_ind = (np.where(ssd_results == ssd_select)[0][0], np.where(ssd_results == ssd_select)[1][0])
            org_strip_up = org_texture[ssd_ind[0]:ssd_ind[0]+strip_w, ssd_ind[1]:ssd_ind[1]+strip_h]

            # find ssd matrix between synthesized_strip_down and org_strip_up
            org_strip_up_gray = cv2.cvtColor(org_strip_up, cv2.COLOR_BGR2GRAY)
            synthesized_strip_down_gray = cv2.cvtColor(np.uint8(synthesized_strip_down), cv2.COLOR_BGR2GRAY)
            strip_ssd = (org_strip_up_gray - synthesized_strip_down_gray) ** 2

            # find horizontal minimum cut for strip_ssd:
            min_cut_ind = find_min_cut(strip_ssd)
            joint_strip = find_joint_strip(synthesized_strip_down, org_strip_up, min_cut_ind)

            # replace joint_strip with synthesized_strip_down:
            synthesized_texture[synthesized_strip_start_h:synthesized_strip_start_h+strip_w, 0:synthesized_strip_start_w+strip_h] = joint_strip
            synthesized_texture[synthesized_strip_start_h+strip_w:synthesized_strip_start_h+patch_h, 0:patch_w] = (
            org_texture[ssd_ind[0]+strip_w:ssd_ind[0]+patch_h, ssd_ind[1]:ssd_ind[1]+patch_w]
            )

        # Synthesis other patchs of synthesized_texture:
        else:
            i_quotient = i // int(patch_count)
            i_res = i % int(patch_count)

            # find L shape strip:
            synthesized_strip_start_h = int((patch_h - strip_w) * i_quotient)
            synthesized_strip_start_w = (patch_w - strip_w) * i_res
            synthesized_strip_full = synthesized_texture[synthesized_strip_start_h:synthesized_strip_start_h+patch_h, synthesized_strip_start_w:synthesized_strip_start_w+patch_w]

            # use template matching with SSD method to find 10 most similar L_strips to synthesized_strip_full with mask
            # produce mask for synthesized_strip_full:
            mask = np.zeros((patch_h, patch_w))
            mask[strip_w:strip_h, strip_w:strip_h] = 0
            mask[0:strip_w, 0:strip_h] = 1
            mask[0:strip_h, 0:strip_w] = 1
            mask = np.uint8(mask)

            ssd_results = cv2.matchTemplate(np.uint8(org_texture), np.uint8(synthesized_strip_full), cv2.TM_SQDIFF, None, mask)
            rand_ind = np.random.randint(1, 10)
            ssd_select = np.sort(ssd_results.flatten())[rand_ind]
            ssd_ind = (np.where(ssd_results == ssd_select)[0][0], np.where(ssd_results == ssd_select)[1][0])
            org_strip_full = org_texture[ssd_ind[0]:ssd_ind[0]+patch_h, ssd_ind[1]:ssd_ind[1]+patch_w]

            # split L shape of synthesized_strip_full and org_strip_full to vertical and horizontal strips:
            synthesized_strip_L_vertical = synthesized_strip_full[0:strip_h, 0:strip_w]
            synthesized_strip_L_horizontal = synthesized_strip_full[0:strip_w, 0:strip_h]

            org_strip_L_vertical = org_strip_full[0:strip_h, 0:strip_w]
            org_strip_L_horizontal = org_strip_full[0:strip_w, 0:strip_h]

            # find ssd matrix between synthesized_strip_Ls and org_strip_Ls:
            synthesized_strip_L_v_gray = cv2.cvtColor(np.uint8(synthesized_strip_L_vertical), cv2.COLOR_BGR2GRAY)
            org_strip_L_v_gray = cv2.cvtColor(np.uint8(org_strip_L_vertical), cv2.COLOR_BGR2GRAY)
            strip_ssd_L_v = (org_strip_L_v_gray - synthesized_strip_L_v_gray) ** 2

            synthesized_strip_L_h_gray = cv2.cvtColor(np.uint8(synthesized_strip_L_horizontal), cv2.COLOR_BGR2GRAY)
            org_strip_L_h_gray = cv2.cvtColor(np.uint8(org_strip_L_horizontal), cv2.COLOR_BGR2GRAY)
            strip_ssd_L_h = (org_strip_L_h_gray - synthesized_strip_L_h_gray) ** 2

            # find vertical and horizontal minimum cut for strip_ssd_L_v and strip_ssd_L_h respectively:
            min_cut_v_ind = find_min_cut(strip_ssd_L_v)
            min_cut_h_ind = find_min_cut(strip_ssd_L_h)

            # find joint_strip_v and joint_strip_h
            joint_strip_v = find_joint_strip(synthesized_strip_L_vertical, org_strip_L_vertical, min_cut_v_ind)
            joint_strip_h = find_joint_strip(synthesized_strip_L_horizontal, org_strip_L_horizontal, min_cut_v_ind)

            # replace joint_strip_v and joint_strip_h with synthesized_strip_L_vertical and synthesized_strip_L_horizontal respectively:
            synthesized_texture[synthesized_strip_start_h:synthesized_strip_start_h+strip_h, synthesized_strip_start_w:synthesized_strip_start_w+strip_w] = joint_strip_v
            synthesized_texture[synthesized_strip_start_h:synthesized_strip_start_h+strip_w, synthesized_strip_start_w:synthesized_strip_start_w+strip_h] = joint_strip_h

            synthesized_texture[synthesized_strip_start_h+strip_w:synthesized_strip_start_h+strip_h, synthesized_strip_start_w+strip_w:synthesized_strip_start_w+patch_w] = (
            org_texture[ssd_ind[0]+strip_w:ssd_ind[0]+patch_h, ssd_ind[1]+strip_w:ssd_ind[1]+patch_w]
            )

    # Crop the output image to size output_size:
    synthesized_output = synthesized_texture[0:output_size, 0:output_size]

    return np.uint8(synthesized_output)

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در ادامه با استفاده تابع <code>subplot</code> دو بافت اصلی و بافت سنتز شده را به دست می آوریم:
</font>
</p>

In [ ]:
def subplot(org_texture, synthesized_texture):
    # add border to org_texture for use cv2.hconcat:
    white = [255,255,255]     # border color
    org_texture_border = cv2.copyMakeBorder(org_texture,synthesized_texture.shape[0] - org_texture.shape[0],0,0,synthesized_texture.shape[1] - org_texture.shape[1],cv2.BORDER_CONSTANT,value=white)
    result = cv2.hconcat((org_texture_border, synthesized_texture))

    return result


In [ ]:
# load original textures
# with BGR space:
org_texture1 = cv2.imread('org_images/texture02.png')
org_texture2 = cv2.imread('org_images/texture06.jpg')
org_texture3 = cv2.imread('org_images/texture-dani.png')
org_texture4 = cv2.imread('org_images/knitting_pattern.jpg')

In [ ]:
# find synthesized_texture of org_textures:
synthesized_texture1 = texture_synthesize(org_texture1)
synthesized_texture2 = texture_synthesize(org_texture2)
synthesized_texture3 = texture_synthesize(org_texture3)
synthesized_texture4 = texture_synthesize(org_texture4)

In [ ]:
# find subplot of original and synthesized textures and save them:
res11 = subplot(org_texture1, synthesized_texture1)
res12 = subplot(org_texture2, synthesized_texture2)
res13 = subplot(org_texture3, synthesized_texture3)
res14 = subplot(org_texture4, synthesized_texture4)

cv2.imwrite('res_images/texture02_result.jpg', res11)
cv2.imwrite('res_images/texture06_result.jpg', res12)
cv2.imwrite('res_images/texture-dani_result.jpg', res13)
cv2.imwrite('res_images/knitting_pattern_result.jpg', res14)

True